<a href="https://colab.research.google.com/github/DCI-alxogm/ml2023-leo0nel09/blob/main/nov3-4/Redneuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import glob
import numpy as np
import copy
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
#Los datos corresponden a un subset del dataset cats vs dogs de https://www.kaggle.com/c/dogs-vs-cats
files=glob.glob('/content/drive/MyDrive/CATS_DOGS/*')

#numero de pixeles para nuestras imágenes.
num_px=64

train_x=[]
train_y=[]
for file in files[:100]:
    img=Image.open(file)
    img=img.resize((num_px,num_px))
    data=np.asarray(img)
    train_x.append(data)
    img.close()
    if 'dog' in file:
        train_y.append(1)
    if 'cat' in file:
        train_y.append(0)
train_y=np.asarray(train_y)
train_x=np.asarray(train_x)

test_x=[]
test_y=[]
for file in files[500:]:
    #print(file)
    img=Image.open(file)
    img=img.resize((num_px,num_px))
    data=np.asarray(img)
    test_x.append(data)
    img.close()
    if 'dog' in file:
        test_y.append(1)
    if 'cat' in file:
        test_y.append(0)
test_y=np.asarray(test_y)
test_x=np.asarray(test_x)

train_x_flatten = train_x.reshape(train_x.shape[0], -1).T

test_x_flatten = test_x.reshape(test_x.shape[0], -1).T

train_set_y=train_y.reshape((1,train_y.shape[0]))
test_set_y=test_y.reshape((1,test_y.shape[0]))



train_set_x= train_x_flatten/255
test_set_x= test_x_flatten/255

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

def inicializar_con_ceros(dim):
    return np.zeros((dim, 1)), 0

def propagacion(w, b, X, Y):
    m = X.shape[1]
    A = sigmoide(np.dot(w.T, X) + b)
    costo = (-1/m) * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
    dw = np.dot(X, (A - Y).T) / m
    db = np.sum(A - Y) / m
    return {"dw": dw, "db": db}, costo

def optimizar(w, b, X, Y, num_iteraciones, tasa_aprendizaje, imprimir_costo=False):
    costos = []
    for i in range(num_iteraciones):
        gradientes, costo = propagacion(w, b, X, Y)
        w -= tasa_aprendizaje * gradientes["dw"]
        b -= tasa_aprendizaje * gradientes["db"]
        if i % 10 == 0:
            costos.append(costo)
        if imprimir_costo and i % 10 == 0:
            print("Costo después de la iteración %i: %f" % (i, costo))
    return {"w": w, "b": b}, gradientes, costos

def predecir(w, b, X):
    A = sigmoide(np.dot(w.T, X) + b)
    return np.where(A > 0.5, 1, 0)

def modelo(train_x, train_y, test_x, test_y, num_iteraciones=2000, tasa_aprendizaje=0.5, imprimir_costo=False):
    train_x_aplanado = train_x.reshape(train_x.shape[0], -1).T
    test_x_aplanado = test_x.reshape(test_x.shape[0], -1).T
    w, b = inicializar_con_ceros(train_x_aplanado.shape[0])
    parametros, gradientes, costos = optimizar(w, b, train_x_aplanado, train_y, num_iteraciones, tasa_aprendizaje, imprimir_costo)
    Y_prediccion_test = predecir(parametros["w"], parametros["b"], test_x_aplanado)
    Y_prediccion_train = predecir(parametros["w"], parametros["b"], train_x_aplanado)
    return {"costos": costos, "Y_prediccion_test": Y_prediccion_test, "Y_prediccion_train": Y_prediccion_train, "w": parametros["w"], "b": parametros["b"], "tasa_aprendizaje": tasa_aprendizaje, "num_iteraciones": num_iteraciones}

d = model(train_x, train_y, test_x, test_y, num_iterations=100, taza_aprendizaje=0.0000005, print_costo=True) # Llama a la función model

Cost after iteration 0: 0.693147
Cost after iteration 10: 3.955771
Cost after iteration 20: 3.526754
Cost after iteration 30: 10.551256
Cost after iteration 40: 1.180591
Cost after iteration 50: 6.541668
Cost after iteration 60: 9.271857
Cost after iteration 70: 8.797126
Cost after iteration 80: 0.427908
Cost after iteration 90: 6.281967
train accuracy: 81.0 %
test accuracy: 60.06216006216006 %
